In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id   
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType, LongType
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1625390255634_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

In [2]:
#song_data/A/B/C/
#s3://udacity-dend/song_data
#s3://udacity-dend/log_data
song_path = "s3://udacity-dend/song_data/A/B/*/*.json"
log_path = "s3://udacity-dend/log_data/2018/*/*.json"
log = spark.read.json(log_path)
song = spark.read.json(song_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
filtered_user_log = log.filter(log.page == "NextSong")
filtered_user_log.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26')

In [4]:
def get_date_unit_function(unit):
    switcher={
        'start_date_time': udf(lambda x: datetime.fromtimestamp(x/ 1000).strftime("%d/%m/%y %H:%M:%S")),
        'start_time': udf(lambda x: datetime.fromtimestamp(x/ 1000).strftime("%H:%M:%S")),
        'hour': udf(lambda x: datetime.fromtimestamp(x/ 1000).hour),
        'day': udf(lambda x: datetime.fromtimestamp(x/ 1000).day),
        'week': udf(lambda x: datetime.fromtimestamp(x/ 1000).isocalendar()[1]),
        'month': udf(lambda x: datetime.fromtimestamp(x/ 1000).month),
        'year': udf(lambda x: datetime.fromtimestamp(x/ 1000).year),
        'weekday': udf(lambda x: datetime.fromtimestamp(x/ 1000).weekday())
    }
    return switcher.get(unit)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
time_schema = StructType([ \
        StructField("start_time",StringType(),True), \
        StructField("hour",IntegerType(),True), \
        StructField("day",IntegerType(),True), \
        StructField("week", IntegerType(), True), \
        StructField("month", IntegerType(), True), \
        StructField("year", IntegerType(), True), \
        StructField("weekday", IntegerType(), True)
      ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
filtered_user_log = filtered_user_log.withColumn("start_time",get_date_unit_function('start_time')(filtered_user_log.ts)). \
                            withColumn("hour",get_date_unit_function('hour')(filtered_user_log.ts)). \
                            withColumn("day",get_date_unit_function('day')(filtered_user_log.ts)). \
                            withColumn("week",get_date_unit_function('week')(filtered_user_log.ts)). \
                            withColumn("month",get_date_unit_function('month')(filtered_user_log.ts)). \
                            withColumn("year",get_date_unit_function('year')(filtered_user_log.ts)). \
                            withColumn("weekday",get_date_unit_function('weekday')(filtered_user_log.ts))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
time_data = filtered_user_log.select("start_time", col("hour").cast('int').alias("hour"), col("day").cast('int').alias("day"), \
                                     col("week").cast('int').alias("week"), col("month").cast('int').alias("month"), \
                                     col("year").cast('int').alias("year"), col("weekday").cast('int').alias("weekday")).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
time_df = spark.createDataFrame(data=time_data, schema=time_schema)
time_df.write.partitionBy("year","month").mode("overwrite").parquet("s3://sparkify-stored-tabled/time-table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
user_schema = StructType([ \
        StructField("user_id",StringType(),False), \
        StructField("first_name",StringType(),True), \
        StructField("last_name",StringType(),True), \
        StructField("gender", StringType(), True), \
        StructField("level", StringType(), True)
      ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
user_data = filtered_user_log.select("userId", "firstName", "lastName", "gender", "level").where(col("userId").isNotNull()).dropDuplicates().collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
user_df = spark.createDataFrame(data=user_data, schema=user_schema)
user_df.write.mode("overwrite").parquet("s3://sparkify-stored-tabled/users-table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
song_schema = StructType([ \
        StructField("song_id",StringType(),False), \
        StructField("title",StringType(),True), \
        StructField("artist_id",StringType(),True), \
        StructField("year", LongType(), True), \
        StructField("duration", DoubleType(), True)
      ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
song_data = song.select("song_id", "title", "artist_id", col("year").cast('int').alias("year"), \
                        col("duration").cast('double').alias("duration")).where(col("song_id").isNotNull()).dropDuplicates().collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
song_df = spark.createDataFrame(data=song_data, schema=song_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
song_df.write.partitionBy("year","artist_id").mode("overwrite").parquet("s3://sparkify-stored-tabled/songs-table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
artist_schema = StructType([ \
        StructField("artist_id",StringType(),False), \
        StructField("name",StringType(),True), \
        StructField("location",StringType(),True), \
        StructField("latitude", DoubleType(), True), \
        StructField("longitude", DoubleType(), True)
      ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
artist_data = song.select("artist_id", col("artist_name").alias("name"), col("artist_location").alias("location"), \
                          col("artist_latitude").alias("latitude"), col("artist_longitude").alias("longitude")).where(col("artist_id").isNotNull()).dropDuplicates().collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
artist_df = spark.createDataFrame(data=artist_data, schema=artist_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
artist_df.write.mode("overwrite").parquet("s3://sparkify-stored-tabled/artists-table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
songplay_schema = StructType([ \
        StructField("songplay_id",LongType(),False), \
        StructField("start_time",LongType(),False), \
        StructField("user_id",StringType(),False), \
        StructField("level", StringType(), True), \
        StructField("song_id", StringType(), True), \
        StructField("artist_id", StringType(), True), \
        StructField("session_id", StringType(), True), \
        StructField("location", StringType(), True), \
        StructField("user_agent", StringType(), True), \
        StructField("year", IntegerType(), False), \
        StructField("month", IntegerType(), False)
      ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
joined_df = filtered_user_log.join(artist_df, filtered_user_log.artist == artist_df.name, how='left').\
                join(song_df, filtered_user_log.song == song_df.title, how='left'). \
                drop(artist_df.name).drop(song_df.title).drop(artist_df.location).drop(artist_df.artist_id). \
                drop(song_df.artist_id). \
                withColumn("songplay_id",monotonically_increasing_id())
joined_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- day: string (nullable = true)
 |-- week: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: string (nullable = true)
 |-- weekday: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |

In [37]:
songplays_data = joined_df.select("songplay_id", col("ts").alias("start_time"), col("userId").alias("user_id"), "level", "song_id", \
                                     "artist_id", col("sessionId").alias("session_id"), \
                                     "location", col("userAgent").alias("user_agent"), get_date_unit_function("year")(joined_df.ts).cast('int'), \
                                     get_date_unit_function("month")(joined_df.ts).cast('int')).dropDuplicates().collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
songplays_df = spark.createDataFrame(data=songplays_data, schema=songplay_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
songplays_df.write.partitionBy("year", "month").mode("overwrite").parquet("s3://sparkify-stored-tabled/songplays-table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
songplays_df.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(songplay_id=6, start_time=1542568680796, user_id='29', level='paid', song_id=None, artist_id=None, session_id='589', location='Atlanta-Sandy Springs-Roswell, GA', user_agent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"', year=2018, month=11), Row(songplay_id=8589934592, start_time=1542130679796, user_id='97', level='paid', song_id=None, artist_id=None, session_id='537', location='Lansing-East Lansing, MI', user_agent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"', year=2018, month=11), Row(songplay_id=8589934604, start_time=1542836964796, user_id='15', level='paid', song_id=None, artist_id=None, session_id='818', location='Chicago-Naperville-Elgin, IL-IN-WI', user_agent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', year=2018, month=11), Row(songplay_id

In [42]:
test_df = spark.read.load("s3://sparkify-stored-tabled/artists-table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
test_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist_id='AR633SY1187B9AC3B9', name='Weezer', location='', latitude=None, longitude=None)